In [1]:
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
#import ruptures as rpt
import warnings
#import ruptures
#import requests
import time
import pickle
import networkx as nx
from tqdm import tqdm
warnings.filterwarnings('ignore')


In [3]:
DATASET_PATH="/media/ll16598/One Touch/INNOVATE/pwc_python_files_from_git/"


In [6]:
import os

def find_files(root, target_filename):
    """Return a list of paths to files in 'root' that match the target filename (case-insensitive)."""
    matches = []
    for dirpath, _, filenames in os.walk(root):
        for file in filenames:
            if file.lower() == target_filename.lower():
                matches.append(os.path.join(dirpath, file))
    return matches

# Dictionaries to hold results: keys are paper titles (folder names)
main_dict = {}
model_dict = {}

# Iterate over the folders in the dataset directory
for paper in os.listdir(DATASET_PATH):
    paper_path = os.path.join(DATASET_PATH, paper)
    if os.path.isdir(paper_path):
        main_files = find_files(paper_path, "main.py")
        model_files = find_files(paper_path, "model.py")
        if main_files:
            main_dict[paper] = main_files
        if model_files:
            model_dict[paper] = model_files

# Print the resulting dictionaries
print("Main Files Dictionary:")
for key, value in main_dict.items():
    print(f"{key}: {value}")

print("\nModel Files Dictionary:")
for key, value in model_dict.items():
    print(f"{key}: {value}")


KeyboardInterrupt: 

In [11]:
from transformers import AutoTokenizer, AutoModel
import torch

# Load GraphCodeBERT tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("microsoft/graphcodebert-base")
model = AutoModel.from_pretrained("microsoft/graphcodebert-base")



tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/539 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at microsoft/graphcodebert-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Embedding shape: (768,)


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [ ]:

def get_graphcodebert_embedding(code: str):
    # Tokenize the input code snippet
    inputs = tokenizer(code, return_tensors="pt", truncation=True, max_length=512)
    # Get the model outputs without gradients
    with torch.no_grad():
        outputs = model(**inputs)
    # Extract the embedding from the [CLS] token
    cls_embedding = outputs.last_hidden_state[:, 0, :]
    return cls_embedding.squeeze().numpy()

# Example usage:
code_snippet = """
def example_function(x):
    return x * 2
"""
embedding = get_graphcodebert_embedding(code_snippet)
print("Embedding shape:", embedding.shape)

In [22]:
import ast
import io
import tokenize

def remove_comments_and_docstrings(source):
    """
    Returns the source code with comments, docstrings, and blank newlines removed.
    This function tokenizes the source and omits tokens corresponding to
    comments and docstrings (when they occur as the first statement in a block).
    It then removes blank lines from the result.
    """
    io_obj = io.StringIO(source)
    output_tokens = []
    prev_toktype = tokenize.INDENT
    last_lineno = -1
    last_col = 0

    try:
        for tok in tokenize.generate_tokens(io_obj.readline):
            token_type, token_string, (srow, scol), (erow, ecol), line = tok
            # Reset column when moving to a new line.
            if srow > last_lineno:
                last_col = 0
            # Skip comment tokens.
            if token_type == tokenize.COMMENT:
                continue
            # Skip string tokens that are likely docstrings.
            if token_type == tokenize.STRING and prev_toktype == tokenize.INDENT:
                continue
            if token_type == tokenize.STRING and last_lineno == 0:
                continue
            # Preserve spacing.
            if scol > last_col:
                output_tokens.append(" " * (scol - last_col))
            output_tokens.append(token_string)
            prev_toktype = token_type
            last_lineno = erow
            last_col = ecol
    except tokenize.TokenError as e:
        print(f"Token error: {e}")
        return source

    # Join the tokens back into a single string.
    cleaned_source = "".join(output_tokens)
    # Remove blank newlines.
    cleaned_lines = [line for line in cleaned_source.splitlines() if line.strip()]
    return "\n".join(cleaned_lines)

def extract_functions(file_content):
    """
    Parse Python source code and extract functions.
    Returns a dictionary where keys are function names and values are the cleaned source code
    of the functions (with comments, docstrings, and blank newlines removed).
    """
    functions = {}
    try:
        tree = ast.parse(file_content)
    except SyntaxError as e:
        print(f"SyntaxError while parsing file: {e}")
        return functions

    for node in ast.walk(tree):
        if isinstance(node, ast.FunctionDef):
            function_source = ast.get_source_segment(file_content, node)
            if function_source:
                cleaned_source = remove_comments_and_docstrings(function_source)
                functions[node.name] = cleaned_source
    return functions

# Example usage:
if __name__ == "__main__":
    sample_code = '''
def example_function(x):
    """
    This docstring should be removed.
    It might span multiple lines.
    """

    # This comment should be removed.

    result = x * 2  # Inline comment also removed.

    return result
'''
    funcs = extract_functions(sample_code)
    for name, code in funcs.items():
        print(f"Function: {name}\n{code}\n")


Function: example_function
def example_function(x):
    result = x * 2                                
    return result



In [23]:
test_mode=True


In [32]:
import ast
import io
import tokenize
import torch
from transformers import AutoTokenizer, AutoModel

def get_graphcodebert_embedding(code: str):
    # Tokenize the input code snippet.
    inputs = tokenizer(code, return_tensors="pt", truncation=True, max_length=512)
    # Get model outputs without gradients.
    with torch.no_grad():
        outputs = model(**inputs)
    # Extract the embedding from the [CLS] token.
    cls_embedding = outputs.last_hidden_state[:, 0, :]
    return cls_embedding.squeeze().numpy()

def remove_comments_and_docstrings(source):
    """
    Returns the source code with comments, docstrings, and blank newlines removed.
    This function tokenizes the source and omits tokens corresponding to
    comments and docstrings (when they occur as the first statement in a block),
    then removes blank lines.
    """
    io_obj = io.StringIO(source)
    output_tokens = []
    prev_toktype = tokenize.INDENT
    last_lineno = -1
    last_col = 0

    try:
        for tok in tokenize.generate_tokens(io_obj.readline):
            token_type, token_string, (srow, scol), (erow, ecol), line = tok
            if srow > last_lineno:
                last_col = 0
            # Skip comments.
            if token_type == tokenize.COMMENT:
                continue
            # Skip string tokens that are likely docstrings.
            if token_type == tokenize.STRING and prev_toktype == tokenize.INDENT:
                continue
            if token_type == tokenize.STRING and last_lineno == 0:
                continue
            if scol > last_col:
                output_tokens.append(" " * (scol - last_col))
            output_tokens.append(token_string)
            prev_toktype = token_type
            last_lineno = erow
            last_col = ecol
    except tokenize.TokenError as e:
        print(f"Token error: {e}")
        return source

    cleaned_source = "".join(output_tokens)
    # Remove blank newlines.
    cleaned_lines = [line for line in cleaned_source.splitlines() if line.strip()]
    return "\n".join(cleaned_lines)

def extract_functions(file_content):
    """
    Extract standalone (module-level) functions from the file content.
    Returns a dictionary mapping function names to their cleaned source code.
    """
    functions = {}
    try:
        tree = ast.parse(file_content)
    except SyntaxError as e:
        print(f"SyntaxError while parsing file: {e}")
        return functions

    # Only consider nodes at the module level.
    for node in tree.body:
        if isinstance(node, ast.FunctionDef):
            function_source = ast.get_source_segment(file_content, node)
            if function_source:
                cleaned_source = remove_comments_and_docstrings(function_source)
                functions[node.name] = cleaned_source
    return functions

def extract_methods(file_content):
    """
    Extract methods (functions within classes) from the file content.
    Returns a dictionary mapping a compound key "ClassName.method_name" to the cleaned source code.
    """
    methods = {}
    try:
        tree = ast.parse(file_content)
    except SyntaxError as e:
        print(f"SyntaxError while parsing file: {e}")
        return methods

    for node in tree.body:
        if isinstance(node, ast.ClassDef):
            class_name = node.name
            for child in node.body:
                if isinstance(child, ast.FunctionDef):
                    method_source = ast.get_source_segment(file_content, child)
                    if method_source:
                        cleaned_source = remove_comments_and_docstrings(method_source)
                        methods[f"{class_name}.{child.name}"] = cleaned_source
    return methods

def extract_classes(file_content):
    """
    Parse Python source code and extract class definitions.
    Returns a dictionary where keys are class names and values are the cleaned source code
    of the classes.
    """
    classes = {}
    try:
        tree = ast.parse(file_content)
    except SyntaxError as e:
        print(f"SyntaxError while parsing file: {e}")
        return classes

    for node in ast.walk(tree):
        if isinstance(node, ast.ClassDef):
            class_source = ast.get_source_segment(file_content, node)
            if class_source:
                cleaned_source = remove_comments_and_docstrings(class_source)
                classes[node.name] = cleaned_source
    return classes


def get_clean_document(file_content):
    """
    Returns the entire file content cleaned (removing comments, docstrings, and blank lines).
    """
    return remove_comments_and_docstrings(file_content)

# Example usage:
if __name__ == "__main__":
    sample_code = '''
"""
Module-level docstring that should be removed.
"""

# Module-level comment to be removed.

def example_function(x):
    """
    Function-level docstring to be removed.
    """
    # This is a comment.
    result = x * 2  # Inline comment.
    return result

class ExampleClass:
    """
    This class docstring should be removed.
    """
    def method_one(self, y):
        # Comment inside method.
        return y + 1

    def method_two(self, z):
        """Inline docstring in method."""
        return z - 1
'''

    # Extract and clean module-level functions.
    functions = extract_functions(sample_code)
    # Extract and clean methods within classes.
    methods = extract_methods(sample_code)
    
    classes = extract_classes(sample_code)
    # Clean the full document.
    full_document = get_clean_document(sample_code)

    # Get embeddings for standalone functions.
    function_embeddings = {}
    for func_name, func_source in functions.items():
        embedding = get_graphcodebert_embedding(func_source)
        function_embeddings[func_name] = embedding
    print("Function embeddings:")
    for name in function_embeddings:
        print(f"  {name}: shape {function_embeddings[name].shape}")
        
    class_embeddings = {}
    for class_name, class_source in classes.items():
        embedding = get_graphcodebert_embedding(class_source)
        class_embeddings[class_name] = embedding
    print("\nClass embeddings:")
    for name in class_embeddings:
        print(f"  {name}: shape {class_embeddings[name].shape}")

    # Get embeddings for class methods.
    method_embeddings = {}
    for method_name, method_source in methods.items():
        embedding = get_graphcodebert_embedding(method_source)
        method_embeddings[method_name] = embedding
    print("\nMethod embeddings:")
    for name in method_embeddings:
        print(f"  {name}: shape {method_embeddings[name].shape}")

    # Get embedding for full document.
    document_embedding = get_graphcodebert_embedding(full_document)
    print(f"\nDocument embedding shape: {document_embedding.shape}")


Function embeddings:
  example_function: shape (768,)

Class embeddings:
  ExampleClass: shape (768,)

Method embeddings:
  ExampleClass.method_one: shape (768,)
  ExampleClass.method_two: shape (768,)

Document embedding shape: (768,)


In [36]:
model_dict

{'AttnGAN__Fine-Grained_Text_to_Image_Generation_with_Attentional_Generative_Adversarial_Networks': ['/media/ll16598/One Touch/INNOVATE/pwc_python_files_from_git/AttnGAN__Fine-Grained_Text_to_Image_Generation_with_Attentional_Generative_Adversarial_Networks/model.py'],
 'StackGAN____Realistic_Image_Synthesis_with_Stacked_Generative_Adversarial_Networks': ['/media/ll16598/One Touch/INNOVATE/pwc_python_files_from_git/StackGAN____Realistic_Image_Synthesis_with_Stacked_Generative_Adversarial_Networks/model.py',
  '/media/ll16598/One Touch/INNOVATE/pwc_python_files_from_git/StackGAN____Realistic_Image_Synthesis_with_Stacked_Generative_Adversarial_Networks/code/model.py'],
 'Online_Deep_Learning__Learning_Deep_Neural_Networks_on_the_Fly': ['/media/ll16598/One Touch/INNOVATE/pwc_python_files_from_git/Online_Deep_Learning__Learning_Deep_Neural_Networks_on_the_Fly/model.py',
  '/media/ll16598/One Touch/INNOVATE/pwc_python_files_from_git/Online_Deep_Learning__Learning_Deep_Neural_Networks_on_the

In [35]:
file_list

['/media/ll16598/One Touch/INNOVATE/pwc_python_files_from_git/AttnGAN__Fine-Grained_Text_to_Image_Generation_with_Attentional_Generative_Adversarial_Networks/model.py']

In [41]:
# Dictionary to store embeddings: { paper_title: { file_path: {function_name: embedding} } }
model_embeddings_function = {}
model_embeddings_class = {}
model_embeddings_method = {}
model_embeddings_document = {}

for paper_title, file_list in tqdm(model_dict.items(), desc="Processing papers"):
    paper_results_function={}
    paper_results_class={}
    paper_results_method={}
    paper_results_document={}
    for f, file_path in enumerate(file_list):
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                content = f.read()
        except Exception as e:
            print(f"Error reading {file_path}: {e}")
            continue        
        
        functions = extract_functions(sample_code)
        methods = extract_methods(sample_code)
        classes = extract_classes(sample_code)
        full_document = get_clean_document(sample_code)

        # Get embeddings for standalone functions.
        function_embeddings = {}
        for func_name, func_source in functions.items():
            embedding = get_graphcodebert_embedding(func_source)
            function_embeddings[func_name] = embedding
        class_embeddings = {}
        for class_name, class_source in classes.items():
            embedding = get_graphcodebert_embedding(class_source)
            class_embeddings[class_name] = embedding
        # Get embeddings for class methods.
        method_embeddings = {}
        for method_name, method_source in methods.items():
            embedding = get_graphcodebert_embedding(method_source)
            method_embeddings[method_name] = embedding

        document_embedding = get_graphcodebert_embedding(full_document)

        paper_results_function[file_path] = function_embeddings
        paper_results_class[file_path] = class_embeddings
        paper_results_method[file_path] = method_embeddings
        paper_results_document[file_path] = document_embedding

        
    model_embeddings_function[paper_title] = paper_results_function
    model_embeddings_class[paper_title] = paper_results_class
    model_embeddings_method[paper_title] = paper_results_method
    model_embeddings_document[paper_title] = paper_results_document



Processing papers:   2%|▍                      | 11/664 [00:09<09:09,  1.19it/s]


KeyboardInterrupt: 

In [ ]:

def get_code_embedding(code: str):
    # Tokenize input code
    inputs = tokenizer(code, return_tensors="pt", truncation=True, max_length=512)
    # Get model outputs without gradients
    with torch.no_grad():
        outputs = model(**inputs)
    # Use the [CLS] token's embedding as the representation
    cls_embedding = outputs.last_hidden_state[:, 0, :]
    return cls_embedding.squeeze().numpy()


In [9]:
model_dict

{'AttnGAN__Fine-Grained_Text_to_Image_Generation_with_Attentional_Generative_Adversarial_Networks': ['/media/ll16598/One Touch/INNOVATE/pwc_python_files_from_git/AttnGAN__Fine-Grained_Text_to_Image_Generation_with_Attentional_Generative_Adversarial_Networks/model.py'],
 'StackGAN____Realistic_Image_Synthesis_with_Stacked_Generative_Adversarial_Networks': ['/media/ll16598/One Touch/INNOVATE/pwc_python_files_from_git/StackGAN____Realistic_Image_Synthesis_with_Stacked_Generative_Adversarial_Networks/model.py',
  '/media/ll16598/One Touch/INNOVATE/pwc_python_files_from_git/StackGAN____Realistic_Image_Synthesis_with_Stacked_Generative_Adversarial_Networks/code/model.py'],
 'Online_Deep_Learning__Learning_Deep_Neural_Networks_on_the_Fly': ['/media/ll16598/One Touch/INNOVATE/pwc_python_files_from_git/Online_Deep_Learning__Learning_Deep_Neural_Networks_on_the_Fly/model.py',
  '/media/ll16598/One Touch/INNOVATE/pwc_python_files_from_git/Online_Deep_Learning__Learning_Deep_Neural_Networks_on_the